In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from datetime import datetime

In [2]:
df = pd.read_excel('recomendation.xlsx')

# First we can need to remove some features 
- because these features is noise and incorrect we need to first drop this feature and create same feature using feature engeenering on the basis of available itemID for better analysis

In [3]:
df=df.drop(columns=['PurchaseHistory','CartActivity','WishlistActivity','AbandonedCartData','BrowsingHistory',"ProductName"])

In [4]:
pd.set_option("display.max_columns",None)
df.sample(2)

,UserID,ItemID,Rating,Clicks,Views,TimeSpentOnItem,PurchaseDate,SessionDuration,DeviceType,Age,Gender,Location,Income,Occupation,SignUpDate,MembershipLevel,Device,TimeOfInteraction,SearchQueries,Category,Price,Discount,Brand,Description,Tags,Color,Size,Stock,Ratings,Reviews,ReleaseDate,PopularityScore
355,U04946,Item00971,5,9,45,278.76,NaT,4.08,Mobile,55,Male,"East Seanport, GE",Medium,Heritage manager,2021-08-09,Regular,Desktop,Evening,['company'],Fashion,373.41,14.61,Smith-Chaney,Himself force plan person order turn many ready.,"['stuff', 'him', 'bad']",Yellow,XL,35,1.9,Live across fear quality better through hit ab...,2024-02-27,1.87
2349,U00458,Item00149,4,5,47,138.36,NaT,93.76,Desktop,60,Male,"East Patricia, SN",Medium,Barrister,2023-09-27,Gold,Desktop,Night,"['less', 'quickly', 'office', 'though']",Home Decor,352.31,9.61,"Carroll, Thompson and Schneider",Garden upon consumer light product a car deal.,"['human', 'event', 'hospital']",Black,L,14,2.8,System civil religious life each production la...,2020-05-15,15.73


# Now we can create new feature
- Now we can create a new features 'CartActivity', 'WishlistActivity', 'AbandonedCartData', 'BrowsingHistory' on the basis of particular users .

In [5]:
# Extract unique ItemIDs for random sampling
unique_item_ids = df['ItemID'].unique()

# Function to assign a random list of ItemIDs
def assign_random_items(item_list, n=3):
    return list(np.random.choice(item_list, size=n, replace=False))

# Create new columns with random ItemIDs

df['CartActivity'] = df['UserID'].apply(lambda _: assign_random_items(unique_item_ids))
df['WishlistActivity'] = df['UserID'].apply(lambda _: assign_random_items(unique_item_ids))
df['AbandonedCartData'] = df['UserID'].apply(lambda _: assign_random_items(unique_item_ids))
df['BrowsingHistory'] = df['UserID'].apply(lambda _: assign_random_items(unique_item_ids))

# Display the first few rows to verify
df[['UserID', 'CartActivity', 'WishlistActivity', 'AbandonedCartData', 'BrowsingHistory']].head()

,UserID,CartActivity,WishlistActivity,AbandonedCartData,BrowsingHistory
0,U03449,"[Item00002, Item00386, Item00043]","[Item00779, Item00879, Item00931]","[Item00229, Item00955, Item00834]","[Item00475, Item00983, Item00345]"
1,U01475,"[Item00535, Item00263, Item00800]","[Item00181, Item00363, Item00018]","[Item00152, Item00417, Item00260]","[Item00346, Item00587, Item00700]"
2,U02136,"[Item00210, Item00068, Item00987]","[Item00787, Item00648, Item00197]","[Item00130, Item00911, Item00512]","[Item00182, Item00824, Item00746]"
3,U00332,"[Item00503, Item00897, Item00780]","[Item00426, Item00872, Item00028]","[Item00992, Item00007, Item00413]","[Item00984, Item00299, Item00908]"
4,U04194,"[Item00902, Item00524, Item00377]","[Item00658, Item00525, Item00609]","[Item00583, Item00248, Item00981]","[Item00424, Item00046, Item00625]"


# New feature Engeenering
- now we can create a new feature PurchaseHistory on the basis of a particular user past purchase available on dataset.

In [6]:
# Group by 'UserID' and collect all 'ItemID's for each user as a list
purchase_history_df = df.groupby('UserID')['ItemID'].apply(list).reset_index()

# Rename the column to 'PurchaseHistory' for clarity
purchase_history_df.rename(columns={'ItemID': 'PurchaseHistory'}, inplace=True)

# Merge this back to the original DataFrame if you need to retain all original data with the new feature
df = df.merge(purchase_history_df, on='UserID', how='left')

# Display the final DataFrame with the new 'PurchaseHistory' column
df.head()

,UserID,ItemID,Rating,Clicks,Views,TimeSpentOnItem,PurchaseDate,SessionDuration,DeviceType,Age,Gender,Location,Income,Occupation,SignUpDate,MembershipLevel,Device,TimeOfInteraction,SearchQueries,Category,Price,Discount,Brand,Description,Tags,Color,Size,Stock,Ratings,Reviews,ReleaseDate,PopularityScore,CartActivity,WishlistActivity,AbandonedCartData,BrowsingHistory,PurchaseHistory
0,U03449,Item00047,2,3,42,280.46,NaT,49.98,Desktop,20,Female,"Lake Kristophermouth, NO",Medium,Educational psychologist,2023-07-11,Gold,Desktop,Afternoon,"['model', 'recent', 'training', 'through', 'co...",Fashion,504.25,8.25,"Ramos, Johnson and Bell",Find Mrs family guess camera image decide able...,['event'],White,M,24,3.9,Happen so late space nothing fill mouth health...,2020-09-22,38.13,"[Item00002, Item00386, Item00043]","[Item00779, Item00879, Item00931]","[Item00229, Item00955, Item00834]","[Item00475, Item00983, Item00345]","[Item00047, Item00580]"
1,U01475,Item00360,3,19,8,286.83,NaT,95.71,Mobile,49,Female,"Castanedaside, NL",Medium,Educational psychologist,2022-04-09,Regular,Desktop,Morning,"['control', 'politics', 'agree', 'fund']",Fashion,205.13,1.99,Hall-Spencer,Research seat suggest story career interesting...,['market'],Blue,S,139,1.6,Establish book write education score its hot y...,2021-11-20,5.44,"[Item00535, Item00263, Item00800]","[Item00181, Item00363, Item00018]","[Item00152, Item00417, Item00260]","[Item00346, Item00587, Item00700]",[Item00360]
2,U02136,Item00685,4,10,45,73.91,NaT,51.57,Desktop,23,Female,"Lake Katrinamouth, GA",Medium,Medical secretary,2021-11-25,Regular,Mobile,Afternoon,"['than', 'together']",Electronics,902.47,9.01,"Young, Johnston and Becker",Than available marriage before paper bank nati...,"['major', 'usually', 'choose', 'idea', 'give']",Red,L,116,3.2,Three detail college sing rule able law finall...,2019-11-14,18.93,"[Item00210, Item00068, Item00987]","[Item00787, Item00648, Item00197]","[Item00130, Item00911, Item00512]","[Item00182, Item00824, Item00746]","[Item00685, Item00478]"
3,U00332,Item00623,5,3,39,242.73,NaT,47.61,Desktop,33,Female,"Butlerton, MR",Low,"Psychologist, clinical",2022-12-18,Gold,Mobile,Afternoon,"['their', 'action', 'charge', 'effect', 'during']",Fashion,486.18,0.61,Smith-Williams,Her available teach degree teacher management ...,"['on', 'word', 'continue', 'could']",Blue,NaN,167,4.8,House end already set stay go fine support stu...,2024-08-30,23.07,"[Item00503, Item00897, Item00780]","[Item00426, Item00872, Item00028]","[Item00992, Item00007, Item00413]","[Item00984, Item00299, Item00908]",[Item00623]
4,U04194,Item00815,3,1,30,186.62,2024-09-13 01:21:00,102.16,Mobile,48,Male,"Port Diamond, MD",Low,Financial manager,2020-08-11,Regular,Desktop,Morning,"['perhaps', 'deep', 'tell']",Home Decor,267.90,22.34,Mccall LLC,Officer consider himself leave power maybe.,['outside'],Yellow,M,162,4.1,Wide check quickly short simply hour across ho...,2022-03-15,27.24,"[Item00902, Item00524, Item00377]","[Item00658, Item00525, Item00609]","[Item00583, Item00248, Item00981]","[Item00424, Item00046, Item00625]","[Item00815, Item00705]"


In [7]:
# According to our Literature Review we do not need of Purchase date so we can decidede to drop this feature
df.drop('PurchaseDate', axis=1, inplace=True)


- We have decided to drop the Purchase Date feature as it does not contribute significantly to the objectives of this analysis. Based on the literature review, the focus is on customer behavior and product preferences, and temporal details like the purchase date are not relevant for the current model. Removing this feature helps streamline the dataset and improve analysis efficiency.

In [8]:
df.isnull().sum()

UserID                 0
ItemID                 0
Rating                 0
Clicks                 0
Views                  0
TimeSpentOnItem        0
SessionDuration        0
DeviceType             0
Age                    0
Gender                 0
Location               0
Income                 0
Occupation             0
SignUpDate             0
MembershipLevel        0
Device                 0
TimeOfInteraction      0
SearchQueries          0
Category               0
Price                  0
Discount               0
Brand                  0
Description            0
Tags                   0
Color                  0
Size                 946
Stock                  0
Ratings                0
Reviews                0
ReleaseDate            0
PopularityScore        0
CartActivity           0
WishlistActivity       0
AbandonedCartData      0
BrowsingHistory        0
PurchaseHistory        0
dtype: int64

# Filling null values
- There are null values present in size feature so we can decided to fill this null values randomlly.

In [9]:
# we can fill the nan values
df['Size'] = df['Size'].apply(lambda x: np.random.choice(df['Size'].dropna()) if pd.isna(x) else x)


In [10]:
df.isnull().sum()

UserID               0
ItemID               0
Rating               0
Clicks               0
Views                0
TimeSpentOnItem      0
SessionDuration      0
DeviceType           0
Age                  0
Gender               0
Location             0
Income               0
Occupation           0
SignUpDate           0
MembershipLevel      0
Device               0
TimeOfInteraction    0
SearchQueries        0
Category             0
Price                0
Discount             0
Brand                0
Description          0
Tags                 0
Color                0
Size                 0
Stock                0
Ratings              0
Reviews              0
ReleaseDate          0
PopularityScore      0
CartActivity         0
WishlistActivity     0
AbandonedCartData    0
BrowsingHistory      0
PurchaseHistory      0
dtype: int64

# Encode Categorical Variables
- For categorical features like Gender, Location, and MembershipLevel, we can use LabelEncoder is used for simplicity.

In [11]:
label_encoders = {}
for col in ['Gender','Location','MembershipLevel','Occupation','DeviceType','Category','Brand','Color','Size']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

# Convert Date Columns to Numeric
- We can Extract relevant features from PurchaseDate, SignUpDate, and ReleaseDate, like the year, month, day, or day of the week, which may affect purchasing behavior.

In [12]:
date_columns = ['SignUpDate','ReleaseDate']

for col in date_columns:
    df[col + '_Year'] = df[col].dt.year
    df[col + '_Month'] = df[col].dt.month
    df[col + '_Day'] = df[col].dt.day
    df[col + '_DayOfWeek'] = df[col].dt.dayofweek

df.drop(columns = date_columns,inplace = True)

In [13]:
df.head(1)

,UserID,ItemID,Rating,Clicks,Views,TimeSpentOnItem,SessionDuration,DeviceType,Age,Gender,Location,Income,Occupation,MembershipLevel,Device,TimeOfInteraction,SearchQueries,Category,Price,Discount,Brand,Description,Tags,Color,Size,Stock,Ratings,Reviews,PopularityScore,CartActivity,WishlistActivity,AbandonedCartData,BrowsingHistory,PurchaseHistory,SignUpDate_Year,SignUpDate_Month,SignUpDate_Day,SignUpDate_DayOfWeek,ReleaseDate_Year,ReleaseDate_Month,ReleaseDate_Day,ReleaseDate_DayOfWeek
0,U03449,Item00047,2,3,42,280.46,49.98,0,20,0,1148,Medium,184,0,Desktop,Afternoon,"['model', 'recent', 'training', 'through', 'co...",2,504.25,8.25,693,Find Mrs family guess camera image decide able...,['event'],4,1,24,3.9,Happen so late space nothing fill mouth health...,38.13,"[Item00002, Item00386, Item00043]","[Item00779, Item00879, Item00931]","[Item00229, Item00955, Item00834]","[Item00475, Item00983, Item00345]","[Item00047, Item00580]",2023,7,11,1,2020,9,22,1


# Scale Numerical Columns
- we can Scale numerical columns to a similar range using MinMaxScaler

In [14]:
scaler = MinMaxScaler()
numerical_columns = ['Clicks', 'Views', 'TimeSpentOnItem', 'SessionDuration', 'Age', 'Price', 'Discount', 'Stock', 'Ratings', 'PopularityScore']

df[numerical_columns] = scaler.fit_transform(df[numerical_columns])


In [15]:
scaler = MinMaxScaler()
numerical_columns = ['Clicks','Views','TimeSpentOnItem','SessionDuration','Age','Price','Discount','Stock','Ratings','PopularityScore']

df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

In [16]:
df.sample()

,UserID,ItemID,Rating,Clicks,Views,TimeSpentOnItem,SessionDuration,DeviceType,Age,Gender,Location,Income,Occupation,MembershipLevel,Device,TimeOfInteraction,SearchQueries,Category,Price,Discount,Brand,Description,Tags,Color,Size,Stock,Ratings,Reviews,PopularityScore,CartActivity,WishlistActivity,AbandonedCartData,BrowsingHistory,PurchaseHistory,SignUpDate_Year,SignUpDate_Month,SignUpDate_Day,SignUpDate_DayOfWeek,ReleaseDate_Year,ReleaseDate_Month,ReleaseDate_Day,ReleaseDate_DayOfWeek
331,U04097,Item00026,2,0.736842,0.77551,0.584003,0.045252,0,0.470588,1,324,Medium,509,1,Mobile,Night,"['sense', 'whom']",2,0.646524,0.418139,265,Road size positive stand tax could including e...,"['similar', 'but', 'marriage', 'accept', 'bag']",1,2,0.130653,0.525,Research hotel participant woman reason moveme...,0.151327,"[Item00433, Item00219, Item00844]","[Item00899, Item00525, Item00453]","[Item00358, Item00212, Item00012]","[Item00820, Item00971, Item00132]",[Item00026],2022,3,20,6,2021,4,9,4


In [17]:
df[['CartActivity','WishlistActivity','AbandonedCartData','BrowsingHistory']].head(3)

,CartActivity,WishlistActivity,AbandonedCartData,BrowsingHistory
0,"[Item00002, Item00386, Item00043]","[Item00779, Item00879, Item00931]","[Item00229, Item00955, Item00834]","[Item00475, Item00983, Item00345]"
1,"[Item00535, Item00263, Item00800]","[Item00181, Item00363, Item00018]","[Item00152, Item00417, Item00260]","[Item00346, Item00587, Item00700]"
2,"[Item00210, Item00068, Item00987]","[Item00787, Item00648, Item00197]","[Item00130, Item00911, Item00512]","[Item00182, Item00824, Item00746]"


In [18]:
df.to_csv("new_df")

In [19]:
df.sample(1)

,UserID,ItemID,Rating,Clicks,Views,TimeSpentOnItem,SessionDuration,DeviceType,Age,Gender,Location,Income,Occupation,MembershipLevel,Device,TimeOfInteraction,SearchQueries,Category,Price,Discount,Brand,Description,Tags,Color,Size,Stock,Ratings,Reviews,PopularityScore,CartActivity,WishlistActivity,AbandonedCartData,BrowsingHistory,PurchaseHistory,SignUpDate_Year,SignUpDate_Month,SignUpDate_Day,SignUpDate_DayOfWeek,ReleaseDate_Year,ReleaseDate_Month,ReleaseDate_Day,ReleaseDate_DayOfWeek
1319,U02146,Item00358,4,0.789474,0.510204,0.847758,0.438473,0,0.980392,0,1171,Medium,501,2,Mobile,Evening,['movement'],3,0.396397,0.273091,195,Than whatever lay key answer interest good rev...,"['public', 'activity', 'family']",5,2,0.045226,0.375,Cultural daughter strong program draw as singl...,0.561042,"[Item00523, Item00198, Item00707]","[Item00286, Item00123, Item00627]","[Item00653, Item00670, Item00822]","[Item00292, Item00428, Item00756]","[Item00358, Item00196]",2021,6,6,6,2023,12,9,5


# Process Text Data (Tags, Description)
- we can Convert text-based data like Tags and Description into a vectorized form for content-based filtering. we can use CountVectorizer or TF-IDF Vectorizer for this.

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorize Tags
tfidf_tags = TfidfVectorizer(max_features=100)
tags_matrix = tfidf_tags.fit_transform(df['Tags']).toarray()

# Add vectorized tags to the Dataframe
for i in range(tags_matrix.shape[1]):
    df[f'Tag_{i}'] = tags_matrix[:,i]

# Vectorize Description
tfidf_description = TfidfVectorizer(max_features=200)
description_matrix = tfidf_description.fit_transform(df['Description']).toarray()

# Add vectorized description to the Dataframe
for i in range(description_matrix.shape[1]):
    df[f'Description_{i}'] = description_matrix[:,i]

# Drop original text columns if not needed
df.drop(columns=['Tags','Description'],inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_12196\1690066049.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Tag_{i}'] = tags_matrix[:,i]
C:\Users\HP\AppData\Local\Temp\ipykernel_12196\1690066049.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Tag_{i}'] = tags_matrix[:,i]
C:\Users\HP\AppData\Local\Temp\ipykernel_12196\1690066049.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at onc

In [21]:
df.sample()

,UserID,ItemID,Rating,Clicks,Views,TimeSpentOnItem,SessionDuration,DeviceType,Age,Gender,Location,Income,Occupation,MembershipLevel,Device,TimeOfInteraction,SearchQueries,Category,Price,Discount,Brand,Color,Size,Stock,Ratings,Reviews,PopularityScore,CartActivity,WishlistActivity,AbandonedCartData,BrowsingHistory,PurchaseHistory,SignUpDate_Year,SignUpDate_Month,SignUpDate_Day,SignUpDate_DayOfWeek,ReleaseDate_Year,ReleaseDate_Month,ReleaseDate_Day,ReleaseDate_DayOfWeek,Tag_0,Tag_1,Tag_2,Tag_3,Tag_4,Tag_5,Tag_6,Tag_7,Tag_8,Tag_9,Tag_10,Tag_11,Tag_12,Tag_13,Tag_14,Tag_15,Tag_16,Tag_17,Tag_18,Tag_19,Tag_20,Tag_21,Tag_22,Tag_23,Tag_24,Tag_25,Tag_26,Tag_27,Tag_28,Tag_29,Tag_30,Tag_31,Tag_32,Tag_33,Tag_34,Tag_35,Tag_36,Tag_37,Tag_38,Tag_39,Tag_40,Tag_41,Tag_42,Tag_43,Tag_44,Tag_45,Tag_46,Tag_47,Tag_48,Tag_49,Tag_50,Tag_51,Tag_52,Tag_53,Tag_54,Tag_55,Tag_56,Tag_57,Tag_58,Tag_59,Tag_60,Tag_61,Tag_62,Tag_63,Tag_64,Tag_65,Tag_66,Tag_67,Tag_68,Tag_69,Tag_70,Tag_71,Tag_72,Tag_73,Tag_74,Tag_75,Tag_76,Tag_77,Tag_78,Tag_79,Tag_80,Tag_81,Tag_82,Tag_83,Tag_84,Tag_85,Tag_86,Tag_87,Tag_88,Tag_89,Tag_90,Tag_91,Tag_92,Tag_93,Tag_94,Tag_95,Tag_96,Tag_97,Tag_98,Tag_99,Description_0,Description_1,Description_2,Description_3,Description_4,Description_5,Description_6,Description_7,Description_8,Description_9,Description_10,Description_11,Description_12,Description_13,Description_14,Description_15,Description_16,Description_17,Description_18,Description_19,Description_20,Description_21,Description_22,Description_23,Description_24,Description_25,Description_26,Description_27,Description_28,Description_29,Description_30,Description_31,Description_32,Description_33,Description_34,Description_35,Description_36,Description_37,Description_38,Description_39,Description_40,Description_41,Description_42,Description_43,Description_44,Description_45,Description_46,Description_47,Description_48,Description_49,Description_50,Description_51,Description_52,Description_53,Description_54,Description_55,Description_56,Description_57,Description_58,Description_59,Description_60,Description_61,Description_62,Description_63,Description_64,Description_65,Description_66,Description_67,Description_68,Description_69,Description_70,Description_71,Description_72,Description_73,Description_74,Description_75,Description_76,Description_77,Description_78,Description_79,Description_80,Description_81,Description_82,Description_83,Description_84,Description_85,Description_86,Description_87,Description_88,Description_89,Description_90,Description_91,Description_92,Description_93,Description_94,Description_95,Description_96,Description_97,Description_98,Description_99,Description_100,Description_101,Description_102,Description_103,Description_104,Description_105,Description_106,Description_107,Description_108,Description_109,Description_110,Description_111,Description_112,Description_113,Description_114,Description_115,Description_116,Description_117,Description_118,Description_119,Description_120,Description_121,Description_122,Description_123,Description_124,Description_125,Description_126,Description_127,Description_128,Description_129,Description_130,Description_131,Description_132,Description_133,Description_134,Description_135,Description_136,Description_137,Description_138,Description_139,Description_140,Description_141,Description_142,Description_143,Description_144,Description_145,Description_146,Description_147,Description_148,Description_149,Description_150,Description_151,Description_152,Description_153,Description_154,Description_155,Description_156,Description_157,Description_158,Description_159,Description_160,Description_161,Description_162,Description_163,Description_164,Description_165,Description_166,Description_167,Description_168,Description_169,Description_170,Description_171,Description_172,Description_173,Description_174,Description_175,Description_176,Description_177,Description_178,Description_179,Description_180,Description_181,Description_182,Description_183,Desc

In [22]:
df.shape

(5000, 340)

# Convert  Reviews to Numeric.
- we can Reviews  contain text, use NLP techniques to extract sentiment. 




In [23]:
pip install textblob


In [24]:
from textblob import TextBlob

# Define a function to get sentiment polarity
def sentiment_analysis_function(review):
    analysis = TextBlob(review)
    return analysis.sentiment.polarity


In [25]:
# Apply the sentiment function to each review
df['Reviews_Sentiment'] = df['Reviews'].apply(lambda x: sentiment_analysis_function(x) if pd.notnull(x) else 0)

# Drop the original Reviews column
df.drop(columns=['Reviews'], inplace=True)


C:\Users\HP\AppData\Local\Temp\ipykernel_12196\3442079452.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Reviews_Sentiment'] = df['Reviews'].apply(lambda x: sentiment_analysis_function(x) if pd.notnull(x) else 0)


In [26]:
df.sample(1)

,UserID,ItemID,Rating,Clicks,Views,TimeSpentOnItem,SessionDuration,DeviceType,Age,Gender,Location,Income,Occupation,MembershipLevel,Device,TimeOfInteraction,SearchQueries,Category,Price,Discount,Brand,Color,Size,Stock,Ratings,PopularityScore,CartActivity,WishlistActivity,AbandonedCartData,BrowsingHistory,PurchaseHistory,SignUpDate_Year,SignUpDate_Month,SignUpDate_Day,SignUpDate_DayOfWeek,ReleaseDate_Year,ReleaseDate_Month,ReleaseDate_Day,ReleaseDate_DayOfWeek,Tag_0,Tag_1,Tag_2,Tag_3,Tag_4,Tag_5,Tag_6,Tag_7,Tag_8,Tag_9,Tag_10,Tag_11,Tag_12,Tag_13,Tag_14,Tag_15,Tag_16,Tag_17,Tag_18,Tag_19,Tag_20,Tag_21,Tag_22,Tag_23,Tag_24,Tag_25,Tag_26,Tag_27,Tag_28,Tag_29,Tag_30,Tag_31,Tag_32,Tag_33,Tag_34,Tag_35,Tag_36,Tag_37,Tag_38,Tag_39,Tag_40,Tag_41,Tag_42,Tag_43,Tag_44,Tag_45,Tag_46,Tag_47,Tag_48,Tag_49,Tag_50,Tag_51,Tag_52,Tag_53,Tag_54,Tag_55,Tag_56,Tag_57,Tag_58,Tag_59,Tag_60,Tag_61,Tag_62,Tag_63,Tag_64,Tag_65,Tag_66,Tag_67,Tag_68,Tag_69,Tag_70,Tag_71,Tag_72,Tag_73,Tag_74,Tag_75,Tag_76,Tag_77,Tag_78,Tag_79,Tag_80,Tag_81,Tag_82,Tag_83,Tag_84,Tag_85,Tag_86,Tag_87,Tag_88,Tag_89,Tag_90,Tag_91,Tag_92,Tag_93,Tag_94,Tag_95,Tag_96,Tag_97,Tag_98,Tag_99,Description_0,Description_1,Description_2,Description_3,Description_4,Description_5,Description_6,Description_7,Description_8,Description_9,Description_10,Description_11,Description_12,Description_13,Description_14,Description_15,Description_16,Description_17,Description_18,Description_19,Description_20,Description_21,Description_22,Description_23,Description_24,Description_25,Description_26,Description_27,Description_28,Description_29,Description_30,Description_31,Description_32,Description_33,Description_34,Description_35,Description_36,Description_37,Description_38,Description_39,Description_40,Description_41,Description_42,Description_43,Description_44,Description_45,Description_46,Description_47,Description_48,Description_49,Description_50,Description_51,Description_52,Description_53,Description_54,Description_55,Description_56,Description_57,Description_58,Description_59,Description_60,Description_61,Description_62,Description_63,Description_64,Description_65,Description_66,Description_67,Description_68,Description_69,Description_70,Description_71,Description_72,Description_73,Description_74,Description_75,Description_76,Description_77,Description_78,Description_79,Description_80,Description_81,Description_82,Description_83,Description_84,Description_85,Description_86,Description_87,Description_88,Description_89,Description_90,Description_91,Description_92,Description_93,Description_94,Description_95,Description_96,Description_97,Description_98,Description_99,Description_100,Description_101,Description_102,Description_103,Description_104,Description_105,Description_106,Description_107,Description_108,Description_109,Description_110,Description_111,Description_112,Description_113,Description_114,Description_115,Description_116,Description_117,Description_118,Description_119,Description_120,Description_121,Description_122,Description_123,Description_124,Description_125,Description_126,Description_127,Description_128,Description_129,Description_130,Description_131,Description_132,Description_133,Description_134,Description_135,Description_136,Description_137,Description_138,Description_139,Description_140,Description_141,Description_142,Description_143,Description_144,Description_145,Description_146,Description_147,Description_148,Description_149,Description_150,Description_151,Description_152,Description_153,Description_154,Description_155,Description_156,Description_157,Description_158,Description_159,Description_160,Description_161,Description_162,Description_163,Description_164,Description_165,Description_166,Description_167,Description_168,Description_169,Description_170,Description_171,Description_172,Description_173,Description_174,Description_175,Description_176,Description_177,Description_178,Description_179,Description_180,Description_181,Description_182,Description_183,Description_

In [27]:
df[['Income','Device','TimeOfInteraction']]

,Income,Device,TimeOfInteraction
0,Medium,Desktop,Afternoon
1,Medium,Desktop,Morning
2,Medium,Mobile,Afternoon
3,Low,Mobile,Afternoon
4,Low,Desktop,Morning
...,...,...,...
4995,Medium,Desktop,Evening
4996,Low,Desktop,Evening
4997,Medium,Mobile,Evening
4998,Medium,Mobile,Night


# Encoding features

In [28]:
# Ordinal encoding for 'Income'
income_order = {'Low': 1, 'Medium': 2, 'High': 3}
df['Income'] = df['Income'].map(income_order)

# Perform one-hot encoding on 'Device' and 'TimeOfInteraction'
df = pd.get_dummies(df, columns=['Device', 'TimeOfInteraction'], prefix=['Device', 'Time'])



In [29]:
df.sample(1)

,UserID,ItemID,Rating,Clicks,Views,TimeSpentOnItem,SessionDuration,DeviceType,Age,Gender,Location,Income,Occupation,MembershipLevel,SearchQueries,Category,Price,Discount,Brand,Color,Size,Stock,Ratings,PopularityScore,CartActivity,WishlistActivity,AbandonedCartData,BrowsingHistory,PurchaseHistory,SignUpDate_Year,SignUpDate_Month,SignUpDate_Day,SignUpDate_DayOfWeek,ReleaseDate_Year,ReleaseDate_Month,ReleaseDate_Day,ReleaseDate_DayOfWeek,Tag_0,Tag_1,Tag_2,Tag_3,Tag_4,Tag_5,Tag_6,Tag_7,Tag_8,Tag_9,Tag_10,Tag_11,Tag_12,Tag_13,Tag_14,Tag_15,Tag_16,Tag_17,Tag_18,Tag_19,Tag_20,Tag_21,Tag_22,Tag_23,Tag_24,Tag_25,Tag_26,Tag_27,Tag_28,Tag_29,Tag_30,Tag_31,Tag_32,Tag_33,Tag_34,Tag_35,Tag_36,Tag_37,Tag_38,Tag_39,Tag_40,Tag_41,Tag_42,Tag_43,Tag_44,Tag_45,Tag_46,Tag_47,Tag_48,Tag_49,Tag_50,Tag_51,Tag_52,Tag_53,Tag_54,Tag_55,Tag_56,Tag_57,Tag_58,Tag_59,Tag_60,Tag_61,Tag_62,Tag_63,Tag_64,Tag_65,Tag_66,Tag_67,Tag_68,Tag_69,Tag_70,Tag_71,Tag_72,Tag_73,Tag_74,Tag_75,Tag_76,Tag_77,Tag_78,Tag_79,Tag_80,Tag_81,Tag_82,Tag_83,Tag_84,Tag_85,Tag_86,Tag_87,Tag_88,Tag_89,Tag_90,Tag_91,Tag_92,Tag_93,Tag_94,Tag_95,Tag_96,Tag_97,Tag_98,Tag_99,Description_0,Description_1,Description_2,Description_3,Description_4,Description_5,Description_6,Description_7,Description_8,Description_9,Description_10,Description_11,Description_12,Description_13,Description_14,Description_15,Description_16,Description_17,Description_18,Description_19,Description_20,Description_21,Description_22,Description_23,Description_24,Description_25,Description_26,Description_27,Description_28,Description_29,Description_30,Description_31,Description_32,Description_33,Description_34,Description_35,Description_36,Description_37,Description_38,Description_39,Description_40,Description_41,Description_42,Description_43,Description_44,Description_45,Description_46,Description_47,Description_48,Description_49,Description_50,Description_51,Description_52,Description_53,Description_54,Description_55,Description_56,Description_57,Description_58,Description_59,Description_60,Description_61,Description_62,Description_63,Description_64,Description_65,Description_66,Description_67,Description_68,Description_69,Description_70,Description_71,Description_72,Description_73,Description_74,Description_75,Description_76,Description_77,Description_78,Description_79,Description_80,Description_81,Description_82,Description_83,Description_84,Description_85,Description_86,Description_87,Description_88,Description_89,Description_90,Description_91,Description_92,Description_93,Description_94,Description_95,Description_96,Description_97,Description_98,Description_99,Description_100,Description_101,Description_102,Description_103,Description_104,Description_105,Description_106,Description_107,Description_108,Description_109,Description_110,Description_111,Description_112,Description_113,Description_114,Description_115,Description_116,Description_117,Description_118,Description_119,Description_120,Description_121,Description_122,Description_123,Description_124,Description_125,Description_126,Description_127,Description_128,Description_129,Description_130,Description_131,Description_132,Description_133,Description_134,Description_135,Description_136,Description_137,Description_138,Description_139,Description_140,Description_141,Description_142,Description_143,Description_144,Description_145,Description_146,Description_147,Description_148,Description_149,Description_150,Description_151,Description_152,Description_153,Description_154,Description_155,Description_156,Description_157,Description_158,Description_159,Description_160,Description_161,Description_162,Description_163,Description_164,Description_165,Description_166,Description_167,Description_168,Description_169,Description_170,Description_171,Description_172,Description_173,Description_174,Description_175,Description_176,Description_177,Description_178,Description_179,Description_180,Description_181,Description_182,Description_183,Description_184,Description_185,Descr

In [30]:
df[['Device_Desktop','Device_Mobile','Device_Tablet','Time_Afternoon','Time_Evening','Time_Morning','Time_Night']]

,Device_Desktop,Device_Mobile,Device_Tablet,Time_Afternoon,Time_Evening,Time_Morning,Time_Night
0,True,False,False,True,False,False,False
1,True,False,False,False,False,True,False
2,False,True,False,True,False,False,False
3,False,True,False,True,False,False,False
4,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...
4995,True,False,False,False,True,False,False
4996,True,False,False,False,True,False,False
4997,False,True,False,False,True,False,False
4998,False,True,False,False,False,False,True


In [31]:
# List of columns you want to convert
columns_to_convert = ['Device_Desktop', 'Device_Mobile', 'Device_Tablet', 
                      'Time_Afternoon', 'Time_Evening', 'Time_Morning', 'Time_Night']

# Convert only the selected columns from boolean to numeric (True -> 1, False -> 0)
df[columns_to_convert] = df[columns_to_convert].astype(int)


In [32]:
df[['Device_Desktop','Device_Mobile','Device_Tablet','Time_Afternoon','Time_Evening','Time_Morning','Time_Night']]

,Device_Desktop,Device_Mobile,Device_Tablet,Time_Afternoon,Time_Evening,Time_Morning,Time_Night
0,1,0,0,1,0,0,0
1,1,0,0,0,0,1,0
2,0,1,0,1,0,0,0
3,0,1,0,1,0,0,0
4,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...
4995,1,0,0,0,1,0,0
4996,1,0,0,0,1,0,0
4997,0,1,0,0,1,0,0
4998,0,1,0,0,0,0,1


In [33]:
df.sample(1)

,UserID,ItemID,Rating,Clicks,Views,TimeSpentOnItem,SessionDuration,DeviceType,Age,Gender,Location,Income,Occupation,MembershipLevel,SearchQueries,Category,Price,Discount,Brand,Color,Size,Stock,Ratings,PopularityScore,CartActivity,WishlistActivity,AbandonedCartData,BrowsingHistory,PurchaseHistory,SignUpDate_Year,SignUpDate_Month,SignUpDate_Day,SignUpDate_DayOfWeek,ReleaseDate_Year,ReleaseDate_Month,ReleaseDate_Day,ReleaseDate_DayOfWeek,Tag_0,Tag_1,Tag_2,Tag_3,Tag_4,Tag_5,Tag_6,Tag_7,Tag_8,Tag_9,Tag_10,Tag_11,Tag_12,Tag_13,Tag_14,Tag_15,Tag_16,Tag_17,Tag_18,Tag_19,Tag_20,Tag_21,Tag_22,Tag_23,Tag_24,Tag_25,Tag_26,Tag_27,Tag_28,Tag_29,Tag_30,Tag_31,Tag_32,Tag_33,Tag_34,Tag_35,Tag_36,Tag_37,Tag_38,Tag_39,Tag_40,Tag_41,Tag_42,Tag_43,Tag_44,Tag_45,Tag_46,Tag_47,Tag_48,Tag_49,Tag_50,Tag_51,Tag_52,Tag_53,Tag_54,Tag_55,Tag_56,Tag_57,Tag_58,Tag_59,Tag_60,Tag_61,Tag_62,Tag_63,Tag_64,Tag_65,Tag_66,Tag_67,Tag_68,Tag_69,Tag_70,Tag_71,Tag_72,Tag_73,Tag_74,Tag_75,Tag_76,Tag_77,Tag_78,Tag_79,Tag_80,Tag_81,Tag_82,Tag_83,Tag_84,Tag_85,Tag_86,Tag_87,Tag_88,Tag_89,Tag_90,Tag_91,Tag_92,Tag_93,Tag_94,Tag_95,Tag_96,Tag_97,Tag_98,Tag_99,Description_0,Description_1,Description_2,Description_3,Description_4,Description_5,Description_6,Description_7,Description_8,Description_9,Description_10,Description_11,Description_12,Description_13,Description_14,Description_15,Description_16,Description_17,Description_18,Description_19,Description_20,Description_21,Description_22,Description_23,Description_24,Description_25,Description_26,Description_27,Description_28,Description_29,Description_30,Description_31,Description_32,Description_33,Description_34,Description_35,Description_36,Description_37,Description_38,Description_39,Description_40,Description_41,Description_42,Description_43,Description_44,Description_45,Description_46,Description_47,Description_48,Description_49,Description_50,Description_51,Description_52,Description_53,Description_54,Description_55,Description_56,Description_57,Description_58,Description_59,Description_60,Description_61,Description_62,Description_63,Description_64,Description_65,Description_66,Description_67,Description_68,Description_69,Description_70,Description_71,Description_72,Description_73,Description_74,Description_75,Description_76,Description_77,Description_78,Description_79,Description_80,Description_81,Description_82,Description_83,Description_84,Description_85,Description_86,Description_87,Description_88,Description_89,Description_90,Description_91,Description_92,Description_93,Description_94,Description_95,Description_96,Description_97,Description_98,Description_99,Description_100,Description_101,Description_102,Description_103,Description_104,Description_105,Description_106,Description_107,Description_108,Description_109,Description_110,Description_111,Description_112,Description_113,Description_114,Description_115,Description_116,Description_117,Description_118,Description_119,Description_120,Description_121,Description_122,Description_123,Description_124,Description_125,Description_126,Description_127,Description_128,Description_129,Description_130,Description_131,Description_132,Description_133,Description_134,Description_135,Description_136,Description_137,Description_138,Description_139,Description_140,Description_141,Description_142,Description_143,Description_144,Description_145,Description_146,Description_147,Description_148,Description_149,Description_150,Description_151,Description_152,Description_153,Description_154,Description_155,Description_156,Description_157,Description_158,Description_159,Description_160,Description_161,Description_162,Description_163,Description_164,Description_165,Description_166,Description_167,Description_168,Description_169,Description_170,Description_171,Description_172,Description_173,Description_174,Description_175,Description_176,Description_177,Description_178,Description_179,Description_180,Description_181,Description_182,Description_183,Description_184,Description_185,Descr

In [34]:
df.to_excel('Preprocessed_recommendation.xlsx', index=False)